In [420]:
using DataFrames
using CSV
using PyPlot

In [421]:
include("./src/coupling.jl")
include("./src/latt_para.jl")

lattice_params! (generic function with 1 method)

# Physical setup

In [422]:
function phy!()
    # average mass of baryons unit: GeV
    mN = (938.272 + 939.566) / 2
    mΣ = (1189.37 + 1192.64 + 1197.45) / 3
    mΛ = 1115.68
    mΞ = (1314.8 + 1321.3) / 2

    # average mass of mesons unit: MeV
    mKbar = (493.68 + 497.65) / 2
    mπ = (139.570 * 2 + 134.977) / 3
    mK = mKbar
    mη = 547.51
    mch = [(mπ, mΣ), (mKbar, mN), (mη, mΛ), (mK, mΞ)]
    decons = [93, 93, 93, 93] .* 1.15
    threshold = [sum(mch[i]) for i in 1:4]
    pp = Dict(
        :mch => mch,
        :decons_vec => decons,
        :threshold => threshold
    )
    return pp
end
physetup = phy!()
aphy = [-2, -1.84, -2.25, -2.52];

In [423]:
phypH = pole_LO!([1430e0, 10e0], physetup, aphy, n=4)
phypL = pole_LO!([1390e0, 40e0], physetup, aphy, n=4)
phypH, phypL

([1428.9069069147067, 13.760136753623309], [1396.653498965157, 74.23171485172188])

In [424]:
df_coupling_LO!(phypL, physetup, aphy, n=4)

Row,gs11,gs22,gs33,gs44
,Float64,Float64,Float64,Float64
1,8.18866,3.79248,0.459575,0.367381


The coupling of higher pole coupled to 4 channels, respectively.

In [425]:
df_coupling_LO!(phypH, physetup, aphy, n=4)

Row,gs11,gs22,gs33,gs44
,Float64,Float64,Float64,Float64
1,1.83988,6.52125,1.70481,0.0903532


# Lattice setup

In [426]:
latt_para = lattice_params!()
params = copy(latt_para);

In [427]:
fitted_para = DataFrame(CSV.File("./out/fitted_para.csv") )
af2ch = fitted_para.para[1:2]
fϕ = fitted_para.para[3]
f = fitted_para.para[4]
params[:decons_vec] = [fϕ, fϕ, fϕ, fϕ] .* f
params

Dict{Symbol, Any} with 5 entries:
  :m_meson    => (203.655, 486.366)
  :threshold  => [1397.59, 1466.14, 1683.94, 1781.53]
  :decons_vec => [106.947, 106.947, 106.947, 106.947]
  :L          => 0.0205243
  :mch        => [(203.655, 1193.94), (486.366, 979.777), (551.1, 1132.84), (48…

Two poles in 2-coupled channel with hadrpn mass from lattice setup

In [428]:
p1405_2ch = DataFrame(CSV.File("./out/Lambda1405_error_band.csv"))

Row,pH,pH_lower_upper,pL,pL_lower_upper
,Float64,String,Float64,String
1,1463.13,"(21.076521562710695, 17.55789502822131)",1409.9,"(44.51012235524445, 9.077572444172802)"
2,17.5122,"(16.278629061908426, 16.823951650463254)",56.789,"(56.788973124854216, 23.091262829504963)"


Coupling of two poles coupled to $\pi\Sigma$ and $\bar{K}N$, respectively.

- Lower pole

In [429]:
df_coupling_LO!(p1405_2ch.pL, params, af2ch)

Row,gs11,gs22
,Float64,Float64
1,10.1181,5.80658


- Higher pole

In [430]:
df_coupling_LO!(p1405_2ch.pH, params, af2ch)

Row,gs11,gs22
,Float64,Float64
1,2.91414,7.43438


Two poles in 4-coupled channel scattering with hadron masses from lattice setup. The subtract costants of channel $\pi\Sigma$ and $\bar{K}N$ are set to their natural values.

In [431]:
a4ch = [af2ch..., -2e0, -2e0];

In [432]:
p1405_4ch = DataFrame(CSV.File("./out/best_pole_LO_4ch_a_phy.csv"))

Row,pH,pH_lower_upper,pL,pL_lower_upper
,Float64,String,Float64,String
1,1457.49,"(24.3944933775183, 20.38022336623544)",1411.7,"(56.354936013300176, 10.141913798039468)"
2,18.2591,"(12.811770356194032, 18.319934679633413)",58.7049,"(58.704925665729704, 23.426219108345606)"


- Lower pole

In [433]:
df_coupling_LO!(p1405_4ch.pL, params, a4ch, n=4)

Row,gs11,gs22,gs33,gs44
,Float64,Float64,Float64,Float64
1,10.5387,5.85103,0.653671,0.431321


- Higher pole

In [434]:
df = df_coupling_LO!(p1405_4ch.pH, params, a4ch, n=4)

Row,gs11,gs22,gs33,gs44
,Float64,Float64,Float64,Float64
1,3.06409,7.91722,2.3849,0.120044


# Coupling within 1-$\sigma$. Two-channel case

In [435]:
fitted_para_1sigma = DataFrame(CSV.File("./out/resonable_1σ_fitted_params.csv") );

We calculate the $\Lambda(1405)$ coupled to $\pi\Sigma$ and $\bar{K}N$ within 1-$\sigma$.

In [436]:
coupling_1sigma_LO!(params, fitted_para_1sigma)

# Coupling with 1-$\sigma$. Four-channel case

We take the natural values of subtract constants for channel $\eta\Lambda$ and $K\Xi$.

In [439]:
coupling_1sigma_LO!(params, fitted_para_1sigma, n=4, a34=[-2.25, -2.52])